# KDD Cup 1999 Data

http://kdd.ics.uci.edu/databases/kddcup99/kddcup99.html

In [2]:
import sklearn
import pandas as pd
from sklearn import preprocessing
from sklearn.utils import resample
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
import numpy as np
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
import time
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.externals import joblib
from sklearn.utils import resample

In [3]:
print('The scikit-learn version is {}.'.format(sklearn.__version__))

The scikit-learn version is 0.18.1.


In [4]:
col_names = ["duration","protocol_type","service","flag","src_bytes",
             "dst_bytes","land","wrong_fragment","urgent","hot","num_failed_logins",
             "logged_in","num_compromised","root_shell","su_attempted","num_root","num_file_creations",
             "num_shells","num_access_files","num_outbound_cmds","is_host_login","is_guest_login","count",
             "srv_count","serror_rate","srv_serror_rate","rerror_rate","srv_rerror_rate","same_srv_rate",
             "diff_srv_rate","srv_diff_host_rate","dst_host_count","dst_host_srv_count",
             "dst_host_same_srv_rate","dst_host_diff_srv_rate","dst_host_same_src_port_rate",
             "dst_host_srv_diff_host_rate","dst_host_serror_rate","dst_host_srv_serror_rate",
             "dst_host_rerror_rate","dst_host_srv_rerror_rate","label"]

In [5]:
data = pd.read_csv("kddcup.data_10_percent", header=None, names = col_names)

# 前処理
## カテゴリ化

In [6]:
data.label.value_counts()

smurf.              280790
neptune.            107201
normal.              97278
back.                 2203
satan.                1589
ipsweep.              1247
portsweep.            1040
warezclient.          1020
teardrop.              979
pod.                   264
nmap.                  231
guess_passwd.           53
buffer_overflow.        30
land.                   21
warezmaster.            20
imap.                   12
rootkit.                10
loadmodule.              9
ftp_write.               8
multihop.                7
phf.                     4
perl.                    3
spy.                     2
Name: label, dtype: int64

In [7]:
data['label2'] = data.label.where(data.label.str.contains('normal'),'atack')

In [8]:
data.label2.value_counts()

atack      396743
normal.     97278
Name: label2, dtype: int64

In [9]:
data['label3'] = data.label.copy()

In [10]:
data.loc[data.label.str.contains('back|land|neptune|pod|smurf|teardrop'),'label3'] = 'DoS'

In [11]:
data.loc[data.label.str.contains('buffer_overflow|loadmodule|perl|rootkit'),'label3'] = 'U2R'

In [12]:
data.loc[data.label.str.contains('ftp_write|guess_passwd|imap|multihop|phf|spy|warezclient|warezmaster'),'label3'] = 'R2L'

In [13]:
data.loc[data.label.str.contains('ipsweep|nmap|portsweep|satan'),'label3'] = 'Probe'

In [14]:
data.label3.value_counts()

DoS        391458
normal.     97278
Probe        4107
R2L          1126
U2R            52
Name: label3, dtype: int64

## サンプリング

In [15]:
data = resample(data,n_samples=2000,random_state=0)

In [16]:
data.shape

(2000, 44)

## 数値化

In [17]:
le_protocol_type = preprocessing.LabelEncoder()

In [18]:
le_protocol_type.fit(data.protocol_type)

LabelEncoder()

In [19]:
data.protocol_type=le_protocol_type.transform(data.protocol_type)

In [20]:
le_service = preprocessing.LabelEncoder()

In [21]:
le_service.fit(data.service)

LabelEncoder()

In [22]:
data.service = le_service.transform(data.service)

In [23]:
le_flag = preprocessing.LabelEncoder()

In [24]:
le_flag.fit(data.flag)

LabelEncoder()

In [25]:
data.flag = le_flag.transform(data.flag)

In [26]:
data.describe()

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_count,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate
count,2000.000000,2000.000000,2000.000000,2000.000000,2.000000e+03,2000.000000,2000.0,2000.00000,2000.0,2000.000000,...,2000.0000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000
mean,62.213500,0.488500,12.034000,3.599000,3.435584e+03,606.287000,0.0,0.00750,0.0,0.012000,...,231.9170,189.213500,0.755880,0.032405,0.594735,0.007305,0.175355,0.174320,0.058020,0.056780
std,712.957848,0.591644,5.784857,0.953758,1.148363e+05,5315.351494,0.0,0.14985,0.0,0.154493,...,65.6843,105.264259,0.409118,0.115545,0.482721,0.043859,0.379430,0.379119,0.230094,0.228411
min,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.0,0.00000,0.0,0.000000,...,1.0000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,8.000000,4.000000,4.400000e+01,0.000000,0.0,0.00000,0.0,0.000000,...,255.0000,66.000000,0.460000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,8.000000,4.000000,5.200000e+02,0.000000,0.0,0.00000,0.0,0.000000,...,255.0000,255.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.000000,1.000000,19.000000,4.000000,1.032000e+03,0.000000,0.0,0.00000,0.0,0.000000,...,255.0000,255.000000,1.000000,0.040000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,14744.000000,2.000000,32.000000,5.000000,5.133876e+06,202976.000000,0.0,3.00000,0.0,3.000000,...,255.0000,255.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [27]:
data.shape

(2000, 44)

## ラベルの分離

In [28]:
y_train_1 = data.label.copy() 

In [29]:
y_train_2 = data.label2.copy()

In [30]:
y_train_3 = data.label3.copy()

In [31]:
x_train = data.drop(['label','label2','label3'],axis=1)

In [32]:
x_train.shape

(2000, 41)

In [33]:
y_train_1.shape

(2000,)

In [34]:
y_train_2.shape

(2000,)

In [35]:
y_train_3.shape

(2000,)

## 標準化

In [36]:
ss = preprocessing.StandardScaler()

In [37]:
ss.fit(x_train)

StandardScaler(copy=True, with_mean=True, with_std=True)

In [38]:
x_train = ss.transform(x_train)

In [39]:
col_names2 = ["duration","protocol_type","service","flag","src_bytes",
             "dst_bytes","land","wrong_fragment","urgent","hot","num_failed_logins",
             "logged_in","num_compromised","root_shell","su_attempted","num_root","num_file_creations",
             "num_shells","num_access_files","num_outbound_cmds","is_host_login","is_guest_login","count",
             "srv_count","serror_rate","srv_serror_rate","rerror_rate","srv_rerror_rate","same_srv_rate",
             "diff_srv_rate","srv_diff_host_rate","dst_host_count","dst_host_srv_count",
             "dst_host_same_srv_rate","dst_host_diff_srv_rate","dst_host_same_src_port_rate",
             "dst_host_srv_diff_host_rate","dst_host_serror_rate","dst_host_srv_serror_rate",
             "dst_host_rerror_rate","dst_host_srv_rerror_rate"]

In [40]:
pd.DataFrame(x_train,columns=col_names2).describe()

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_count,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate
count,2.000000e+03,2.000000e+03,2.000000e+03,2.000000e+03,2.000000e+03,2.000000e+03,2000.0,2.000000e+03,2000.0,2.000000e+03,...,2.000000e+03,2.000000e+03,2.000000e+03,2.000000e+03,2.000000e+03,2.000000e+03,2.000000e+03,2.000000e+03,2.000000e+03,2.000000e+03
mean,1.274536e-16,1.367795e-16,-4.882761e-16,-2.858824e-16,3.262668e-17,3.601286e-18,0.0,2.375808e-16,0.0,-2.817746e-16,...,-1.356137e-16,-1.768585e-16,-9.126033e-17,1.248168e-16,3.029799e-16,-4.483913e-17,-1.521006e-16,1.263434e-16,1.183498e-16,2.871592e-16
std,1.000250e+00,1.000250e+00,1.000250e+00,1.000250e+00,1.000250e+00,1.000250e+00,0.0,1.000250e+00,0.0,1.000250e+00,...,1.000250e+00,1.000250e+00,1.000250e+00,1.000250e+00,1.000250e+00,1.000250e+00,1.000250e+00,1.000250e+00,1.000250e+00,1.000250e+00
min,-8.728294e-02,-8.258718e-01,-2.080779e+00,-3.774439e+00,-2.992470e-02,-1.140919e-01,0.0,-5.006262e-02,0.0,-7.769310e-02,...,-3.516438e+00,-1.788457e+00,-1.848048e+00,-2.805248e-01,-1.232354e+00,-1.665965e-01,-4.622693e-01,-4.599179e-01,-2.522213e-01,-2.486495e-01
25%,-8.728294e-02,-8.258718e-01,-6.975122e-01,4.205473e-01,-2.954145e-02,-1.140919e-01,0.0,-5.006262e-02,0.0,-7.769310e-02,...,3.515113e-01,-1.170809e+00,-7.233959e-01,-2.805248e-01,-1.232354e+00,-1.665965e-01,-4.622693e-01,-4.599179e-01,-2.522213e-01,-2.486495e-01
50%,-8.728294e-02,-8.258718e-01,-6.975122e-01,4.205473e-01,-2.539539e-02,-1.140919e-01,0.0,-5.006262e-02,0.0,-7.769310e-02,...,3.515113e-01,6.251215e-01,5.968481e-01,-2.805248e-01,8.397520e-01,-1.665965e-01,-4.622693e-01,-4.599179e-01,-2.522213e-01,-2.486495e-01
75%,-8.728294e-02,8.647562e-01,1.204479e+00,4.205473e-01,-2.093575e-02,-1.140919e-01,0.0,-5.006262e-02,0.0,-7.769310e-02,...,3.515113e-01,6.251215e-01,5.968481e-01,6.574867e-02,8.397520e-01,-1.665965e-01,-4.622693e-01,-4.599179e-01,-2.522213e-01,-2.486495e-01
max,2.059793e+01,2.555384e+00,3.452288e+00,1.469294e+00,4.468728e+01,3.808221e+01,0.0,1.997498e+01,0.0,1.934558e+01,...,3.515113e-01,6.251215e-01,5.968481e-01,8.376311e+00,8.397520e-01,2.263922e+01,2.173922e+00,2.178436e+00,4.094924e+00,4.130525e+00


## 学習

In [41]:
pca = PCA(copy=True, iterated_power='auto', n_components=3, random_state=None,
  svd_solver='auto', tol=0.0, whiten=False)

In [42]:
pca.fit(x_train)

PCA(copy=True, iterated_power='auto', n_components=3, random_state=None,
  svd_solver='auto', tol=0.0, whiten=False)

In [43]:
x_train2 = pca.transform(x_train)

In [44]:
x_train2.shape

(2000, 3)

In [45]:
clf = SVC(C=10.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma=1.0, kernel='poly',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [46]:
t1=time.perf_counter()
clf.fit(x_train2,y_train_2)
t2=time.perf_counter()

In [47]:
print(t2-t1,"秒")

11.065076628001407 秒


## 予測

In [49]:
t1=time.perf_counter()
pred=clf.predict(x_train2)
t2=time.perf_counter()


In [50]:
print(t2-t1,"秒")

0.009521288997348165 秒


In [51]:
print(classification_report(y_train_2, pred))
print(confusion_matrix(y_train_2, pred))

             precision    recall  f1-score   support

      atack       1.00      0.99      0.99      1578
    normal.       0.96      1.00      0.98       422

avg / total       0.99      0.99      0.99      2000

[[1561   17]
 [   0  422]]


## 学習

In [62]:
pca2 = PCA(copy=True, iterated_power='auto', n_components=4, random_state=None,
  svd_solver='auto', tol=0.0, whiten=False)


In [63]:
clf2 = SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma=0.17782794100389229,
  kernel='rbf', max_iter=-1, probability=False, random_state=None,
  shrinking=True, tol=0.001, verbose=False)

In [64]:
pca2.fit(x_train)

PCA(copy=True, iterated_power='auto', n_components=4, random_state=None,
  svd_solver='auto', tol=0.0, whiten=False)

In [65]:
x_train3 = pca.transform(x_train)

In [66]:
t1=time.perf_counter()
clf2.fit(x_train3,y_train_3)
t2=time.perf_counter()

In [67]:
print(t2-t1,"秒")

0.01056874799905927 秒


## 予測

In [59]:
t1=time.perf_counter()
pred2=clf2.predict(x_train3)
t2=time.perf_counter()


In [60]:
print(t2-t1,"秒")

0.0049498820008011535 秒


In [61]:
print(classification_report(y_train_3, pred2))
print(confusion_matrix(y_train_3, pred2))

             precision    recall  f1-score   support

        DoS       1.00      0.99      0.99      1555
      Probe       1.00      0.74      0.85        19
        R2L       0.00      0.00      0.00         4
    normal.       0.96      1.00      0.98       422

avg / total       0.99      0.99      0.99      2000

[[1542    0    0   13]
 [   4   14    0    1]
 [   0    0    0    4]
 [   0    0    0  422]]


/Users/mhaya/.pyenv/versions/anaconda3-4.3.0/lib/python3.6/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
